In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split

from synthetic import generate_dataset


In [11]:
n_obs = 1000
n_features = 100
n_classes = 3

In [40]:
X,y,graph = generate_dataset(n_classes,n_obs,n_features,nb_characteristic_features=3, \
                             signal=2,diffusion_coefficient=0.2,noise=0)

tensor_x = torch.Tensor(X).float()
tensor_y = torch.Tensor(y).long()

dataset = TensorDataset(tensor_x,tensor_y) 
train_dataset,test_dataset = random_split(dataset,[int(len(dataset)*0.7),int(len(dataset)*0.3)])
train_dataloader = DataLoader(train_dataset,shuffle=True,batch_size=10) 
test_dataloader = DataLoader(test_dataset,shuffle=True,batch_size=10)

In [42]:
class MLP(nn.Module):

    def __init__(self,n_features,n_layers,n_classes):
        super(MLP, self).__init__()
        self.layers = []
        for i in range(n_layers):
            self.layers.append(nn.Linear(n_features,n_features))
        self.last_layer = nn.Linear(n_features,n_classes)

    def forward(self, x):
        for layer in self.layers:
            x = F.relu(layer(x))
        x = self.last_layer(x)
        return x

In [43]:
net = MLP(n_features,2,n_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [54]:
#Training
for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    print('[%d] loss: %.3f' % (epoch + 1, running_loss))

print('Finished Training')

[1] loss: 92.973
[2] loss: 92.909
[3] loss: 92.819
[4] loss: 92.754
[5] loss: 92.639
[6] loss: 92.602
[7] loss: 92.499
[8] loss: 92.416
[9] loss: 92.331
[10] loss: 92.247
[11] loss: 92.191
[12] loss: 92.099
[13] loss: 92.031
[14] loss: 91.949
[15] loss: 91.856
[16] loss: 91.800
[17] loss: 91.714
[18] loss: 91.666
[19] loss: 91.585
[20] loss: 91.487
[21] loss: 91.420
[22] loss: 91.354
[23] loss: 91.275
[24] loss: 91.211
[25] loss: 91.117
[26] loss: 91.073
[27] loss: 91.002
[28] loss: 90.925
[29] loss: 90.864
[30] loss: 90.776
[31] loss: 90.718
[32] loss: 90.652
[33] loss: 90.588
[34] loss: 90.539
[35] loss: 90.436
[36] loss: 90.373
[37] loss: 90.313
[38] loss: 90.269
[39] loss: 90.201
[40] loss: 90.118
[41] loss: 90.063
[42] loss: 89.979
[43] loss: 89.921
[44] loss: 89.863
[45] loss: 89.831
[46] loss: 89.766
[47] loss: 89.672
[48] loss: 89.622
[49] loss: 89.553
[50] loss: 89.512
[51] loss: 89.433
[52] loss: 89.381
[53] loss: 89.329
[54] loss: 89.248
[55] loss: 89.212
[56] loss: 89.148
[

In [56]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_dataloader:
        X, labels = data
        outputs = net(X)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network: %d %%' % (100 * correct / total))

Accuracy of the network: 81 %
